## Content Based Filtering (attibute/user preference measured)

In [ ]:
# More_usful_column_missing = Property Category:such as luxury, budget, high-risk, low-risk, etc
# nearby facilitoes, property historical returns

## Data collection and preprocessing

In [4]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import pickle


In [2]:

from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer

import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import functional as F

# from transformers import AutoTokenizer, AutoModel


In [15]:
# Used dataset as of now
df_LM = pd.read_csv("/Users/rasikagulhane/Desktop/real-estate-ai-be/Models/Recommendation_System/Latest_maintable_copy.csv")

df_LM.columns

#zpid	price	priceForHDP	daysOnZillow	price	livingArea	lotAreaValue	homeType	rentZestimate

Index(['bathrooms', 'bedrooms', 'city', 'country', 'currency', 'daysOnZillow',
       'homeStatus', 'homeStatusForHDP', 'homeType', 'imgSrc', 'isFeatured',
       'isNonOwnerOccupied', 'isPreforeclosureAuction', 'isPremierBuilder',
       'isShowcaseListing', 'isUnmappable', 'isZillowOwned', 'latitude',
       'listing_sub_type', 'livingArea', 'longitude', 'lotAreaUnit',
       'lotAreaValue', 'price', 'priceForHDP', 'rentZestimate',
       'shouldHighlight', 'state', 'streetAddress', 'zestimate', 'zipcode',
       'zpid', 'openHouse', 'open_house_info', 'taxAssessedValue', 'unit',
       'newConstructionType', 'datePriceChanged', 'priceChange',
       'priceReduction', 'providerListingID', 'group_type', 'priceSuffix',
       'videoCount', 'isRentalWithBasePrice', 'comingSoonOnMarketDate'],
      dtype='object')

In [16]:
df_LM

,bathrooms,bedrooms,city,country,currency,daysOnZillow,homeStatus,homeStatusForHDP,homeType,imgSrc,...,newConstructionType,datePriceChanged,priceChange,priceReduction,providerListingID,group_type,priceSuffix,videoCount,isRentalWithBasePrice,comingSoonOnMarketDate
0,4.0,4.0,San Francisco,USA,USD,-1,FOR_SALE,FOR_SALE,MULTI_FAMILY,https://photos.zillowstatic.com/fp/41e39f99d83...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,San Francisco,USA,USD,6,FOR_SALE,FOR_SALE,MULTI_FAMILY,https://photos.zillowstatic.com/fp/b960cdc02b2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1.0,San Francisco,USA,USD,-1,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/6e2f46d2127...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,2.0,San Francisco,USA,USD,7,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/d12c0fbf813...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,San Francisco,USA,USD,7,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/2950bb769f4...,...,NEW_CONSTRUCTION_TYPE_OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14481,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/fa19233f277...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14482,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/304f8c1b159...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14483,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/c5ea6a76c92...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14484,4.0,5.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,SINGLE_FAMILY,https://photos.zillowstatic.com/fp/1e147b5ea46...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_risk= pd.read_csv("/Users/rasikagulhane/Desktop/real-estate-ai-be/Models/Recommendation_System/Latest_maintable_risk_rating_score copy.csv")
df_risk

,zpid,price,priceForHDP,daysOnZillow,price.1,livingArea,lotAreaValue,homeType,rentZestimate,Predicted Score (Linear),Linear Risk Level,Predicted Score (GBT),GBT Risk Level,Predicted Score (RF),RF Risk Level,Average Score,Average Risk Level,investment_risk
0,67395850,649000.0,649000.0,-1,649000.0,780.0,6664.680000,CONDO,3150.0,0.393557,Low,1.245085,Low,1.385398,Low,1.080727,Low,1690.723440
1,2146933844,1200000.0,1200000.0,-1,1200000.0,1389.0,0.530000,CONDO,5600.0,0.728219,Low,2.342499,Low,2.575413,Low,2.013764,Low,3119.987165
2,64969426,699000.0,699000.0,-1,699000.0,830.0,0.990519,CONDO,3259.0,0.423223,Low,1.341657,Low,1.491007,Low,1.163268,Low,1817.418895
3,89237599,499000.0,499000.0,-1,499000.0,601.0,5738.000000,CONDO,2963.0,0.302093,Low,0.945619,Low,1.060282,Low,0.825887,Low,1300.101400
4,2075613570,1275000.0,1275000.0,26,1275000.0,1609.0,9600.624000,CONDO,5998.0,0.775039,Low,2.574067,Low,2.741797,Low,2.166300,Low,3319.945312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8415,30913533,299995.0,299995.0,0,299995.0,2132.0,0.580000,SINGLE_FAMILY,2889.0,0.180704,Low,0.542352,Low,0.629236,Low,0.486140,Low,781.679390
8416,2062410340,495000.0,495000.0,0,495000.0,3130.0,6.690000,SINGLE_FAMILY,1350.0,0.299811,Low,0.945619,Low,1.051854,Low,0.821555,Low,1290.354745
8417,38890557,2325000.0,2325000.0,4,2325000.0,2656.0,2000.000000,SINGLE_FAMILY,7072.0,1.413594,Low,4.604495,Low,5.018522,Low,3.930263,Low,6047.068000
8418,10232947,350000.0,350000.0,0,350000.0,2306.0,1991.000000,SINGLE_FAMILY,2144.0,0.211481,Low,0.641013,Low,0.737428,Low,0.570981,Low,913.119900


In [18]:
# Check for duplicate records
duplicates = df_risk[df_risk.duplicated()]

# Remove duplicate records and keep the first occurrence
df_risk1 = df_risk.drop_duplicates()

df_risk1

,zpid,price,priceForHDP,daysOnZillow,price.1,livingArea,lotAreaValue,homeType,rentZestimate,Predicted Score (Linear),Linear Risk Level,Predicted Score (GBT),GBT Risk Level,Predicted Score (RF),RF Risk Level,Average Score,Average Risk Level,investment_risk
0,67395850,649000.0,649000.0,-1,649000.0,780.0,6664.680000,CONDO,3150.0,0.393557,Low,1.245085,Low,1.385398,Low,1.080727,Low,1690.723440
1,2146933844,1200000.0,1200000.0,-1,1200000.0,1389.0,0.530000,CONDO,5600.0,0.728219,Low,2.342499,Low,2.575413,Low,2.013764,Low,3119.987165
2,64969426,699000.0,699000.0,-1,699000.0,830.0,0.990519,CONDO,3259.0,0.423223,Low,1.341657,Low,1.491007,Low,1.163268,Low,1817.418895
3,89237599,499000.0,499000.0,-1,499000.0,601.0,5738.000000,CONDO,2963.0,0.302093,Low,0.945619,Low,1.060282,Low,0.825887,Low,1300.101400
4,2075613570,1275000.0,1275000.0,26,1275000.0,1609.0,9600.624000,CONDO,5998.0,0.775039,Low,2.574067,Low,2.741797,Low,2.166300,Low,3319.945312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8415,30913533,299995.0,299995.0,0,299995.0,2132.0,0.580000,SINGLE_FAMILY,2889.0,0.180704,Low,0.542352,Low,0.629236,Low,0.486140,Low,781.679390
8416,2062410340,495000.0,495000.0,0,495000.0,3130.0,6.690000,SINGLE_FAMILY,1350.0,0.299811,Low,0.945619,Low,1.051854,Low,0.821555,Low,1290.354745
8417,38890557,2325000.0,2325000.0,4,2325000.0,2656.0,2000.000000,SINGLE_FAMILY,7072.0,1.413594,Low,4.604495,Low,5.018522,Low,3.930263,Low,6047.068000
8418,10232947,350000.0,350000.0,0,350000.0,2306.0,1991.000000,SINGLE_FAMILY,2144.0,0.211481,Low,0.641013,Low,0.737428,Low,0.570981,Low,913.119900


In [19]:
# final_df = pd.merge(df_a, df_b[['A', 'B_new','F']], how="left", left_on=['A', 'B'], right_on=['A', 'B_new']);


RE_df = pd.merge(df_LM, df_risk1[['zpid','Average Score', 'Average Risk Level', 'investment_risk','rentZestimate', 'daysOnZillow','price','lotAreaValue', 'livingArea','priceForHDP','homeType']],
                 on =['zpid','rentZestimate','daysOnZillow','price','lotAreaValue','livingArea','priceForHDP','homeType'], how ='left')

In [20]:
RE_df

,bathrooms,bedrooms,city,country,currency,daysOnZillow,homeStatus,homeStatusForHDP,homeType,imgSrc,...,priceReduction,providerListingID,group_type,priceSuffix,videoCount,isRentalWithBasePrice,comingSoonOnMarketDate,Average Score,Average Risk Level,investment_risk
0,4.0,4.0,San Francisco,USA,USD,-1,FOR_SALE,FOR_SALE,MULTI_FAMILY,https://photos.zillowstatic.com/fp/41e39f99d83...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,San Francisco,USA,USD,6,FOR_SALE,FOR_SALE,MULTI_FAMILY,https://photos.zillowstatic.com/fp/b960cdc02b2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1.0,San Francisco,USA,USD,-1,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/6e2f46d2127...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.080727,Low,1690.72344
3,3.0,2.0,San Francisco,USA,USD,7,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/d12c0fbf813...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,San Francisco,USA,USD,7,FOR_SALE,FOR_SALE,CONDO,https://photos.zillowstatic.com/fp/2950bb769f4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14481,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/fa19233f277...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14482,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/304f8c1b159...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14483,2.0,3.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,TOWNHOUSE,https://photos.zillowstatic.com/fp/c5ea6a76c92...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14484,4.0,5.0,Philadelphia,USA,USD,0,FOR_SALE,FOR_SALE,SINGLE_FAMILY,https://photos.zillowstatic.com/fp/1e147b5ea46...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
useful_columns= ['bathrooms','bedrooms','city','homeType','homeStatus','livingArea','lotAreaUnit','lotAreaValue','price','rentZestimate',
                 'state','zipcode','taxAssessedValue','newConstructionType','datePriceChanged','priceChange',  'Average Score',
       'Average Risk Level', 'investment_risk']


In [22]:
df= RE_df[useful_columns]

In [23]:
df

,bathrooms,bedrooms,city,homeType,homeStatus,livingArea,lotAreaUnit,lotAreaValue,price,rentZestimate,state,zipcode,taxAssessedValue,newConstructionType,datePriceChanged,priceChange,Average Score,Average Risk Level,investment_risk
0,4.0,4.0,San Francisco,MULTI_FAMILY,FOR_SALE,3032.0,sqft,2000.000000,1499500,11674.0,CA,94103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,San Francisco,MULTI_FAMILY,FOR_SALE,4004.0,sqft,1746.756000,1950000,3407.0,CA,94103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,1.0,San Francisco,CONDO,FOR_SALE,780.0,sqft,6664.680000,649000,3150.0,CA,94103,688942.0,NaN,NaN,NaN,1.080727,Low,1690.72344
3,3.0,2.0,San Francisco,CONDO,FOR_SALE,2730.0,acres,1.380900,4900000,NaN,CA,94103,2794274.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,San Francisco,CONDO,FOR_SALE,0.0,acres,0.852700,621000,2800.0,CA,94103,NaN,NEW_CONSTRUCTION_TYPE_OTHER,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14481,2.0,3.0,Philadelphia,TOWNHOUSE,FOR_SALE,856.0,sqft,847.000000,170000,1694.0,PA,19121,61100.0,NaN,NaN,NaN,NaN,NaN,NaN
14482,2.0,3.0,Philadelphia,TOWNHOUSE,FOR_SALE,1076.0,sqft,1688.000000,118750,1685.0,PA,19120,84900.0,NaN,NaN,NaN,NaN,NaN,NaN
14483,2.0,3.0,Philadelphia,TOWNHOUSE,FOR_SALE,1266.0,sqft,1620.000000,148000,1999.0,PA,19150,134600.0,NaN,NaN,NaN,NaN,NaN,NaN
14484,4.0,5.0,Philadelphia,SINGLE_FAMILY,FOR_SALE,3800.0,acres,0.258264,575000,2857.0,PA,19131,218100.0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# plt.figure(figsize=[9,7])
# sns.heatmap(df.corr(), annot= True)

## Data Cleaning

In [26]:
df.isnull().sum()

bathrooms               1775
bedrooms                1956
city                       0
homeType                   0
homeStatus                 0
livingArea              1267
lotAreaUnit             1894
lotAreaValue            1894
price                      0
rentZestimate           1689
state                      0
zipcode                    0
taxAssessedValue        3786
newConstructionType    12736
datePriceChanged       10509
priceChange            10509
Average Score           6874
Average Risk Level      6875
investment_risk         6874
dtype: int64

In [27]:
null_columns= []
for i in df.columns:
  if df[i].isnull().sum() > 0:
    null_columns.append(i)

for i in null_columns:
  a= df[i].nunique()
  print(f"{i} = {a}")

bathrooms = 18
bedrooms = 19
livingArea = 3595
lotAreaUnit = 2
lotAreaValue = 4762
rentZestimate = 4705
taxAssessedValue = 8125
newConstructionType = 3
datePriceChanged = 240
priceChange = 852
Average Score = 7538
Average Risk Level = 3
investment_risk = 7537


In [28]:
df.lotAreaUnit.replace(to_replace=np.NaN,value=df.lotAreaUnit.mode()[0],inplace=True)
df.newConstructionType.replace(to_replace=np.NaN,value='no',inplace=True)
df.datePriceChanged.replace(to_replace=np.NaN,value=0,inplace=True)
df.priceChange.replace(to_replace=np.NaN,value=0,inplace=True)
df['Average Risk Level'].replace(to_replace=np.NaN,value='blank',inplace=True)

/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_66833/66029877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.lotAreaUnit.replace(to_replace=np.NaN,value=df.lotAreaUnit.mode()[0],inplace=True)
/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_66833/66029877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.newConstructionType.replace(to_replace=np.NaN,value='no',inplace=True)
/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_66833/66029877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [29]:
df_num=df.select_dtypes(np.number)

df_num.head()

,bathrooms,bedrooms,livingArea,lotAreaValue,price,rentZestimate,zipcode,taxAssessedValue,datePriceChanged,priceChange,Average Score,investment_risk
0,4.0,4.0,3032.0,2000.0000,1499500,11674.0,94103,NaN,0.0,0.0,NaN,NaN
1,0.0,NaN,4004.0,1746.7560,1950000,3407.0,94103,NaN,0.0,0.0,NaN,NaN
2,2.0,1.0,780.0,6664.6800,649000,3150.0,94103,688942.0,0.0,0.0,1.080727,1690.72344
3,3.0,2.0,2730.0,1.3809,4900000,NaN,94103,2794274.0,0.0,0.0,NaN,NaN
4,1.0,1.0,0.0,0.8527,621000,2800.0,94103,NaN,0.0,0.0,NaN,NaN


In [30]:
df_num.isnull().sum()

bathrooms           1775
bedrooms            1956
livingArea          1267
lotAreaValue        1894
price                  0
rentZestimate       1689
zipcode                0
taxAssessedValue    3786
datePriceChanged       0
priceChange            0
Average Score       6874
investment_risk     6874
dtype: int64

### For Numerical Data

In [31]:

# Create KNNImputer instance
knn_imputer = KNNImputer(n_neighbors=2)  # Adjust n_neighbors as needed

# Perform imputation
imputed_data = knn_imputer.fit_transform(df_num)


# transform the dataset
df_num_imputed = pd.DataFrame(imputed_data,columns=df_num.columns)
df_num_imputed.head()


,bathrooms,bedrooms,livingArea,lotAreaValue,price,rentZestimate,zipcode,taxAssessedValue,datePriceChanged,priceChange,Average Score,investment_risk
0,4.0,4.0,3032.0,2000.0000,1499500.0,11674.0,94103.0,388554.0,0.0,0.0,2.526911,3899.603839
1,0.0,3.5,4004.0,1746.7560,1950000.0,3407.0,94103.0,1557744.5,0.0,0.0,3.293025,5072.165913
2,2.0,1.0,780.0,6664.6800,649000.0,3150.0,94103.0,688942.0,0.0,0.0,1.080727,1690.723440
3,3.0,2.0,2730.0,1.3809,4900000.0,2754.5,94103.0,2794274.0,0.0,0.0,8.232332,12614.180175
4,1.0,1.0,0.0,0.8527,621000.0,2800.0,94103.0,542829.0,0.0,0.0,1.025202,1611.355220


In [32]:
df_num_imputed.columns

Index(['bathrooms', 'bedrooms', 'livingArea', 'lotAreaValue', 'price',
       'rentZestimate', 'zipcode', 'taxAssessedValue', 'datePriceChanged',
       'priceChange', 'Average Score', 'investment_risk'],
      dtype='object')

### For Categorical data

In [33]:

df_cat = df.copy()
df_cat.drop(df_num_imputed, axis = 1, inplace=True)
df_cat.head()

,city,homeType,homeStatus,lotAreaUnit,state,newConstructionType,Average Risk Level
0,San Francisco,MULTI_FAMILY,FOR_SALE,sqft,CA,no,blank
1,San Francisco,MULTI_FAMILY,FOR_SALE,sqft,CA,no,blank
2,San Francisco,CONDO,FOR_SALE,sqft,CA,no,Low
3,San Francisco,CONDO,FOR_SALE,acres,CA,no,blank
4,San Francisco,CONDO,FOR_SALE,acres,CA,NEW_CONSTRUCTION_TYPE_OTHER,blank


### Dataset form for further processing

In [47]:
frames = [df_LM['zpid'], df_cat, df_num_imputed]
data = pd.concat(frames, axis=1)

In [48]:
data

,zpid,city,homeType,homeStatus,lotAreaUnit,state,newConstructionType,Average Risk Level,bathrooms,bedrooms,livingArea,lotAreaValue,price,rentZestimate,zipcode,taxAssessedValue,datePriceChanged,priceChange,Average Score,investment_risk
0,2075615600,San Francisco,MULTI_FAMILY,FOR_SALE,sqft,CA,no,blank,4.0,4.0,3032.0,2000.000000,1499500.0,11674.0,94103.0,388554.0,0.0,0.0,2.526911,3899.603839
1,2072037431,San Francisco,MULTI_FAMILY,FOR_SALE,sqft,CA,no,blank,0.0,3.5,4004.0,1746.756000,1950000.0,3407.0,94103.0,1557744.5,0.0,0.0,3.293025,5072.165913
2,67395850,San Francisco,CONDO,FOR_SALE,sqft,CA,no,Low,2.0,1.0,780.0,6664.680000,649000.0,3150.0,94103.0,688942.0,0.0,0.0,1.080727,1690.723440
3,63197299,San Francisco,CONDO,FOR_SALE,acres,CA,no,blank,3.0,2.0,2730.0,1.380900,4900000.0,2754.5,94103.0,2794274.0,0.0,0.0,8.232332,12614.180175
4,2057008465,San Francisco,CONDO,FOR_SALE,acres,CA,NEW_CONSTRUCTION_TYPE_OTHER,blank,1.0,1.0,0.0,0.852700,621000.0,2800.0,94103.0,542829.0,0.0,0.0,1.025202,1611.355220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14481,10456550,Philadelphia,TOWNHOUSE,FOR_SALE,sqft,PA,no,blank,2.0,3.0,856.0,847.000000,170000.0,1694.0,19121.0,61100.0,0.0,0.0,0.259358,435.671250
14482,10466281,Philadelphia,TOWNHOUSE,FOR_SALE,sqft,PA,no,blank,2.0,3.0,1076.0,1688.000000,118750.0,1685.0,19120.0,84900.0,0.0,0.0,0.228433,385.288358
14483,10469599,Philadelphia,TOWNHOUSE,FOR_SALE,sqft,PA,no,blank,2.0,3.0,1266.0,1620.000000,148000.0,1999.0,19150.0,134600.0,0.0,0.0,0.252887,422.531600
14484,10486291,Philadelphia,SINGLE_FAMILY,FOR_SALE,acres,PA,no,blank,4.0,5.0,3800.0,0.258264,575000.0,2857.0,19131.0,218100.0,0.0,0.0,0.946187,1491.238850


In [50]:

data.to_csv('final_data.csv',index=False)

## Feature Engineering

In [34]:


# Select relevant features for content-based filtering
content_features = data[['bathrooms', 'bedrooms', 'livingArea', 'lotAreaValue', 'price', 'rentZestimate', 'taxAssessedValue', 'Average Score', 'investment_risk']]

# Normalize numerical features
scaler = MinMaxScaler()
normalized_content_features = scaler.fit_transform(content_features)

## Applying Model

In [35]:

# Convert categorical attributes to embeddings
embedding_dims = {
    'city': 16,
    'homeType': 4,
    'state': 8,
    'newConstructionType': 2,
    'Average Risk Level': 3
}

# Create embeddings for categorical features using torch
embeddings = {}
for feature, dim in embedding_dims.items():
    unique_values = data[feature].unique()
    embedding = torch.nn.Embedding(len(unique_values), dim)
    embeddings[feature] = embedding

# Convert categorical values to embeddings using torch
embedded_categorical_features = []
for feature, embedding in embeddings.items():
    values = data[feature].values
    indices = [np.where(unique_values == value)[0][0] if value in unique_values else 0 for value in values]
    indices = torch.tensor(indices)
    embedded = embedding(indices)
    embedded_categorical_features.append(embedded.detach().numpy())

embedded_categorical_features = np.concatenate(embedded_categorical_features, axis=1)


In [36]:
# Combine normalized numerical and embedded categorical features
combined_features = pd.DataFrame(normalized_content_features, columns=content_features.columns)
combined_features = pd.concat([combined_features, pd.DataFrame(embedded_categorical_features)], axis=1)


In [37]:

# Calculate cosine similarity between properties
similarities = cosine_similarity(combined_features)

# Get recommendations for a property (e.g., property at index 0)
property_index = 1
similar_properties = similarities[property_index]
sorted_indices = similar_properties.argsort()[::-1]

# Select top recommended property indices (excluding the input property itself)
top_recommendations = sorted_indices[1:10]


In [38]:

# Display top recommended property indices (row no. is diplayed)

print("Top 10 recommended properties:")
print(top_recommendations)

Top 10 recommended properties:
[ 5428   146  5609  5753 11695  5389  5723 10055  1365]


## TRIAL AND CAN USE when reviews or other textual property description added

In [39]:


# Select relevant features for content-based filtering
content_features = data[['bathrooms', 'bedrooms', 'livingArea', 'lotAreaValue', 'price', 'rentZestimate', 'taxAssessedValue']]

# Normalize numerical features
scaler = MinMaxScaler()
normalized_content_features = scaler.fit_transform(content_features)


In [40]:

# Convert categorical attributes to embeddings
embedding_dims = {
    'city': 16,
    'homeType': 4,
    'state': 8,
    'newConstructionType': 2
}

# Create embeddings for categorical features
embeddings = {}
for feature, dim in embedding_dims.items():
    unique_values = data[feature].unique()
    embedding = torch.nn.Embedding(len(unique_values), dim)
    embeddings[feature] = embedding

# Convert categorical values to embeddings
embedded_categorical_features = []
for feature, embedding in embeddings.items():
    values = data[feature].values
    indices = [np.where(unique_values == value)[0][0] if value in unique_values else 0 for value in values]
    indices = torch.tensor(indices)
    embedded = embedding(indices)
    embedded_categorical_features.append(embedded.detach().numpy())

embedded_categorical_features = np.concatenate(embedded_categorical_features, axis=1)


In [46]:

# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

# Tokenize and get BERT embeddings for textual descriptions
textual_descriptions = data['property_description']  # Assuming this column exists in the dataset
encoded_inputs = tokenizer(list(textual_descriptions), padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    bert_outputs = bert_model(**encoded_inputs).last_hidden_state
textual_embeddings = torch.mean(bert_outputs, dim=1)


KeyError: ignored

In [ ]:

# Combine normalized numerical, embedded categorical, and textual embeddings
combined_features = pd.DataFrame(normalized_content_features, columns=content_features.columns)
combined_features = pd.concat([combined_features, pd.DataFrame(embedded_categorical_features)], axis=1)
combined_features = pd.concat([combined_features, pd.DataFrame(textual_embeddings.numpy())], axis=1)

# Calculate cosine similarity between properties
similarities = cosine_similarity(combined_features)

# Get recommendations for a property (e.g., property at index 0)
property_index = 0
similar_properties = similarities[property_index]
sorted_indices = similar_properties.argsort()[::-1]

# Select top recommended property indices (excluding the input property itself)
top_recommendations = sorted_indices[1:6]

# Display top recommended property indices
print("Top recommended properties:")
print(top_recommendations)


In [ ]:
#TFIDF if reviews and other property information mention in text format
'''
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')  # Download punkt tokenizer if needed

# Sample property descriptions
property_descriptions = [
    "Spacious two-bedroom condo with great views.",
    "Charming single-family home in a quiet neighborhood.",
    "Luxury apartment with modern amenities.",
]

# Tokenization and TF-IDF
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(property_descriptions)

# Display TF-IDF matrix
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())


'''